In [2]:
pip install pulp

254.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 34.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pulp import *

# Définir les données d'entrée
equipements = ['E1', 'E2', 'E3', 'E4']  # Liste des équipements à entretenir
corps_metiers = ['CM1', 'CM2', 'CM3']  # Liste des corps de métier
intervalles = {'CM1': 14, 'CM2': 21, 'CM3': 30}  # Intervalles de maintenance par corps de métier (en jours)
temps_travail = {'CM1': 6, 'CM2': 8, 'CM3': 10}  # Temps de travail estimé par corps de métier (en heures)

# Définir le problème d'optimisation linéaire
prob = LpProblem("Maintenance_Preventive", LpMinimize)

# Définir les variables de décision
x = LpVariable.dicts("Maintenance", [(e, c) for e in equipements for c in corps_metiers], 0, 1, LpBinary)

# Définir la fonction objectif
prob += lpSum([x[(e, c)] * temps_travail[c] for e in equipements for c in corps_metiers])

# Contrainte : chaque équipement doit être maintenu au moins une fois pendant la période de planification
for e in equipements:
    prob += lpSum([x[(e, c)] for c in corps_metiers]) == 1

# Contrainte : la maintenance doit être effectuée dans l'intervalle spécifié pour chaque corps de métier
for c in corps_metiers:
    for i in range(0, len(equipements), intervalles[c]):
        prob += lpSum([x[(equipements[e], c)] for e in range(i, min(i+intervalles[c], len(equipements)))]) >= 1

# Résoudre le problème d'optimisation linéaire
prob.solve()

# Afficher les résultats
print("Statut de la résolution : {}".format(LpStatus[prob.status]))
print("Coût total : {} heures".format(round(value(prob.objective), 2)))

# Afficher la planification de maintenance par équipement et par corps de métier
for e in equipements:
    print("Equipement {} :".format(e))
    for c in corps_metiers:
        if x[(e, c)].value() == 1:
            print("- {} : {} heures".format(c, temps_travail[c]))

# Afficher la charge de travail par corps de métier
for c in corps_metiers:
    temps_total = sum([x[(e, c)].value() * temps_travail[c] for e in equipements])
    print("Charge de travail pour le corps de métier {} : {} heures".format(c, temps_total))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /srv/conda/envs/notebook/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/478d4a0aa0cd457498c8836a81190193-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/478d4a0aa0cd457498c8836a81190193-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 73 RHS
At line 81 BOUNDS
At line 94 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 30 - 0.00 seconds
Cgl0004I processed model has 7 rows, 12 columns (12 integer (12 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 30
Cbc0038I Before mini branch and bound, 12 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not imp